## PHYS64 Project Spring 2024
### Lev Gruber

## H-H ground state energy notebook.

Below, see the first part of my PHYS64 project.
The code was adapted (mostly copied but expanded in functionality to go from just finding a single energy to creating a plot at various energies) from https://qiskit-community.github.io/qiskit-nature/tutorials/03_ground_state_solvers.html, with inspiration on the plot/method from the very outdated (so mostly useless for code) video https://www.youtube.com/watch?v=Z-A6G0WVI9w.

In [ ]:
# imports
import qiskit
import numpy as np
import matplotlib.pyplot as plt
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
import csv

# setup qubit mapper
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.mappers import ParityMapper

# setup classical optimizer for VQE
from qiskit_algorithms.optimizers import L_BFGS_B

# For VQE Backend
from qiskit_aer import Aer

# prepare the ground-state solver and run it
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# For classical result computation
from qiskit_algorithms import NumPyMinimumEigensolver

# For VQE primitive  
from qiskit.primitives import Estimator


In [ ]:
# Code inside for loop adapted (basically copied adding comments) from https://qiskit-community.github.io/qiskit-nature/tutorials/03_ground_state_solvers.html
# Taking above code and placing in loop to generate plot of interatomic distance, hartreefock, vs. vqe guess
# Idea from https://www.youtube.com/watch?v=Z-A6G0WVI9w

'''
Preliminary idea is to create a plot of the ground state energies at various interatomic distances,
demonstrating the difference in result between our ansatz (Hartree Fock / HF), exactly calculated, and VQE estimated.

Code loops across defined 'distances' then applies a VQE method to find the ground state energy, and direct methods for HF and exact solving. 
'''
# Define arrays to place results into
exact_energies = []
vqe_energies = []
hf_energies = []
# Define molecule to be tested
molecule = 'H .0 .0 -{0}; H .0 .0 {0}'

distances = np.arange(0.5, 4.5, 0.25)

for i, d in enumerate(distances):
    print('step', i)
    
    # set up H-H for VQE
    driver = PySCFDriver(
    atom = molecule.format(d/2),
    unit=DistanceUnit.ANGSTROM,
    basis='sto3g' #figure out what this is
        # atom='O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0',
        #unit=DistanceUnit.ANGSTROM, charge=0, spin=0, basis='sto3g' <- H2O
    )

    problem = driver.run()
    
    # set up mapper and VQE
    mapper = JordanWignerMapper() #num_particles=problem.num_particles if using parity
    optimizer = L_BFGS_B() #Limited-memory BFGS Bound optimizer, goal to minimize value of differentiable scalar f

    # setup the estimator primitive for the VQE
    estimator = Estimator()

    # setup VQE using Hartree-Fock as ansatz
    from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
    from qiskit_algorithms import VQE
    # define ansatz
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper
        )
    )
    backend = Aer.get_backend('statevector_simulator')
    vqe = VQE(estimator, ansatz, optimizer)
    # ensure that the optimizer starts in the all-zero state which corresponds to the Hartree-Fock starting point
    vqe.initial_point = [0] * ansatz.num_parameters
    algorithm = GroundStateEigensolver(mapper, vqe)

    # Solve VQE 
    electronic_structure_result = algorithm.solve(problem)
    electronic_structure_result.formatting_precision = 6
    
    #Solve for exact result
    numpy_solver = NumPyMinimumEigensolver()
    exact_calc = GroundStateEigensolver(mapper, numpy_solver)
    res = exact_calc.solve(problem)
    
    # Input into arrays to be graphed
    vqe_energies.append(electronic_structure_result.total_energies[0]) # get vqe ground energy
    hf_energies.append(electronic_structure_result.hartree_fock_energy) # get HF ground energy
    exact_energies.append(res.total_energies[0])
    
rows = zip(distances, vqe_energies, exact_energies, hf_energies)
with open('hh_local_4282024.csv', 'w') as f: #INSERTBACKEND_INSERTDISTANCES_INSERTDATE
    writer = csv.writer(f)
    # Write the headers
    writer.writerow(['Distances', 'VQE Energies', 'Exact Energies', 'HF Energies'])
    # Write the data
    for row in rows:
        writer.writerow(row)    
'''
With three arrays, HF, VQE, and Exact, we can plot them against distance to view the accuracy of the VQE method.
'''
plt.plot(distances, hf_energies, label = 'Hartree-Fock')
plt.plot(distances, exact_energies, 'o', label = 'Exact')
plt.plot(distances, vqe_energies, 'x', label = 'VQE')

plt.xlabel(r'Interatomic Distance ($A^\circ$)')
plt.ylabel('Energy')
plt.title('H-H Ground Energy')
plt.legend(loc = 'upper right')

